In [2]:
import pandas as pd
import numpy as np
import polars as pl
import fastexcel

In [14]:
# Read in file
df = pl.read_excel("data/Data_Tables_LGA_Criminal_Incidents_Year_Ending_March_2025.xlsx", 
                   sheet_name="Table 03",
                #    read_options={
                #     "skip_rows": 6,
                #     }
                   )
# Column names change
# df.columns = ["GCCSA code",	"GCCSA name", "SA4 code", "SA4 name", "SA3 code",
#               "SA3 name", "SA2 code", "SA2 name","2023 Pop", "2024 Pop",
#               "2023-24 Change", "2023-24 Change %", "Natural Increase", "Net internal migration",
#               "Net oveseas migration","Area","Population Density"]

In [15]:
df.head()

Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
i64,str,str,i64,str,str,str,str,i64
2025,"""March""","""Alpine""",3691,"""Coral Bank""","""D Public order and security offences""","""D10 Weapons and explosives offences""","""D11 Firearms offences""",1
2025,"""March""","""Alpine""",3691,"""Dederang""","""B Property and deception offences""","""B10 Arson""","""B19 Other fire related offences""",1
2025,"""March""","""Alpine""",3691,"""Dederang""","""B Property and deception offences""","""B30 Burglary/Break and enter""","""B322 Non-residential non-aggravated burglary""",2
2025,"""March""","""Alpine""",3691,"""Dederang""","""B Property and deception offences""","""B40 Theft""","""B42 Steal from a motor vehicle""",1
2025,"""March""","""Alpine""",3691,"""Dederang""","""B Property and deception offences""","""B40 Theft""","""B49 Other theft""",1


In [5]:
pl.Config.set_fmt_str_lengths(50)
pl.Config.set_tbl_rows(30)
df_2024=(
    df
        .filter(pl.col("Year") == 2024)
        # .filter(pl.col("Offence Subdivision") == "B40 Theft")

        .group_by(["Year","Local Government Area","Postcode", "Suburb/Town Name", "Offence Division", 'Offence Subdivision'])
        .agg(
            pl.sum("Incidents Recorded")
        )
        # .filter(pl.col("Local Government Area") == 'Darebin')
        .sort("Incidents Recorded", descending=True)
        .rename({"Incidents Recorded" : "Incidents Recorded 2024"})
        [:,1:7] # select relevant columns
)
df_2024.write_csv("data/df_crime_2024.csv")

In [22]:
pl.Config.set_tbl_rows(100)
# df_2025=
(
    df
        .filter(pl.col("Year") == 2025)
        .filter(pl.col("Suburb/Town Name") == 'Alphington')
        .filter(pl.col("Offence Subdivision") == 'B40 Theft')

        .group_by(["Year","Local Government Area","Postcode", "Suburb/Town Name", "Offence Division", 'Offence Subdivision'])
        .agg(
            pl.sum("Incidents Recorded")
        )

        .sort("Incidents Recorded", descending=True)
        .rename({"Incidents Recorded" : "Incidents Recorded 2025"})
     
        [:,1:9] # select relevant columns
)
# df_2025.write_csv("data/df_crime_2025.csv")

Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Incidents Recorded 2025
str,i64,str,str,str,i64
"""Darebin""",3078,"""Alphington""","""B Property and deception offences""","""B40 Theft""",161
"""Yarra""",3078,"""Alphington""","""B Property and deception offences""","""B40 Theft""",119


In [7]:
pl.Config.set_tbl_rows(20)
df_crime=(
    df_2024
    .join(
        df_2025,
        on = ['Postcode','Suburb/Town Name','Offence Subdivision'],
        how = 'left'
    )
    .select(["Local Government Area","Postcode", "Suburb/Town Name", 
             "Offence Division", 'Offence Subdivision',"Incidents Recorded 2024",
             "Incidents Recorded 2025"])
    # .filter(
    #     pl.col("Local Government Area") == 'Darebin'
    # )
    .with_columns(
        (
        pl.when(pl.col("Incidents Recorded 2024")>10) # do not record pcp change if sample is under 10
        .then(
            ((pl.col("Incidents Recorded 2025")/pl.col("Incidents Recorded 2024")) - 1)*100)
        .otherwise(None)
        )
        .alias("% change")
    )
    .with_columns(
        (
        pl.col("Incidents Recorded 2025")-pl.col("Incidents Recorded 2024")
    ).alias("# change")
    )
    .sort("Incidents Recorded 2024", descending=True)
)

In [ ]:
int(df_crime.filter(pl.col("Local Government Area") == sel_council).filter(pl.col("Offence Subdivision")==subdivision_value).select(pl.col('Incidents Recorded 2025').mean()).item())

In [24]:
sel_council=df_crime.filter(pl.col("Suburb/Town Name") == 'Balwyn').filter(pl.col("Offence Subdivision")=='A20 Assault and related offences').select(pl.col('Local Government Area')).item()
x = int(df_crime.filter(pl.col("Local Government Area") == sel_council).filter(pl.col("Offence Subdivision")=='A20 Assault and related offences').select(pl.col('Incidents Recorded 2025').mean()).item())
x1 = int(df_crime.filter(pl.col("Local Government Area") == sel_council).filter(pl.col("Offence Subdivision")=='A20 Assault and related offences').select(pl.col('# change').mean()).item())


In [26]:
x

36

In [12]:
(
    df_crime
    .filter(
        pl.col("Local Government Area") == 'Darebin'
    )    
    .filter(
        pl.col("Offence Subdivision")=='B40 Theft'
    )
    .sort("Suburb/Town Name")
)

Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Incidents Recorded 2024,Incidents Recorded 2025,% change,# change
str,i64,str,str,str,i64,i64,f64,i64
"""Darebin""",3078,"""Alphington""","""B Property and deception offences""","""B40 Theft""",93,161,73.11828,68
"""Darebin""",3078,"""Alphington""","""B Property and deception offences""","""B40 Theft""",93,119,27.956989,26
"""Darebin""",3083,"""Bundoora""","""B Property and deception offences""","""B40 Theft""",186,745,300.537634,559
"""Darebin""",3083,"""Bundoora""","""B Property and deception offences""","""B40 Theft""",186,345,85.483871,159
"""Darebin""",3083,"""Bundoora""","""B Property and deception offences""","""B40 Theft""",186,199,6.989247,13
"""Darebin""",3078,"""Fairfield""","""B Property and deception offences""","""B40 Theft""",175,281,60.571429,106
"""Darebin""",3078,"""Fairfield""","""B Property and deception offences""","""B40 Theft""",175,41,-76.571429,-134
"""Darebin""",3083,"""Kingsbury""","""B Property and deception offences""","""B40 Theft""",103,139,34.951456,36
"""Darebin""",3085,"""Macleod""","""B Property and deception offences""","""B40 Theft""",19,114,500.0,95
